In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from alexnet import *
from tensorflow.python import debug as tf_debug

In [2]:
def _image_parser (path, label):
    
    # Convert One Hot Label
    one_hot_label = tf.one_hot(label, NUM_CLASSES)
    
    # Read Image
    image_file = tf.read_file(path)
    image_decoded = tf.image.decode_jpeg(image_file, channels=3)
    image_resized = tf.image.resize_images(image_decoded, INPUT_IMAGE_SIZE)
    
    return image_resized, one_hot_label
    

In [3]:
# Misc
LOG_DIR = './log/'
IMGNET_PRE_MODEL = './imagenet_pretrained_20/model.ckpt'

# Training Parameters
MEAN_IMAGE = np.array([104., 117., 124.])
SHUFFLE_BUFFER= 250
LEARNING_RATE = 0.005
MOMENTUM = 0.9
NUM_EPOCHS = 1
NUM_STEPS = 5000
BATCH_SIZE = 128
DISPLAY_STEP = 1
INPUT_IMAGE_SIZE = [227, 227]
NUM_CLASSES = 20
KEEP_RATE = 0.75

In [4]:
# Load Data
train = pd.read_csv('../train20.txt', delimiter=' ', header=None).sample(frac=1, random_state=100) 

In [9]:
model_graph = tf.Graph()
with model_graph.as_default():  

    keep_prob = tf.placeholder(tf.float32)

    # Dataset
    trn_ds = tf.data.Dataset.from_tensor_slices(
        (np.array(train[0]), np.array(train[1]))).map(_image_parser).shuffle(SHUFFLE_BUFFER).repeat().batch(BATCH_SIZE)

    # Dataset iterators
    iterator = trn_ds.make_one_shot_iterator()
    x, y = iterator.get_next()

    # Image Summary
    tf.summary.image('input', x, 10)
    
    # Build Model
    alexnet = My_AlexNet(x=x, keep_rate=keep_prob, 
                         num_classes=NUM_CLASSES, mean_image = MEAN_IMAGE,
                         skip_layers=['fc8'], weights_path='../bvlc_alexnet.npy')
    alexnet.build()

    # Logits and Predictions
    logits = alexnet.logits
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}

    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)) 
        tf.summary.scalar('cross_entropy_loss', loss_op)

    with tf.variable_scope('train'):
        optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=MOMENTUM)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], tf.argmax(y, 1))
        accuracy_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy_op)

    # Global Initializer
    global_init = tf.global_variables_initializer()

    # Merge Summary
    summary = tf.summary.merge_all()

    # Global saver
    saver = tf.train.Saver()

In [ ]:
with tf.Session(graph=model_graph) as sess:
    
    # Debugger
    # sess = tf_debug.TensorBoardDebugWrapperSession(sess, "marr:7000")
    
    # Writer
    writer = tf.summary.FileWriter('./log/model_1', graph=model_graph)
    
    # Run Global Initializer
#     print('Start initializing.')
#     sess.run(global_init)
#     print('Done')
    
    # Restore pretrained model
    print('Load Pretrained model')
    saver.restore(sess, './imagenet_pretrained_20/model.ckpt')
    print('Model restored.')
    
    print('Start training.')
    
    for step  in range(NUM_STEPS):
        
        sess.run(train_op, feed_dict={keep_prob: KEEP_RATE})

        if step % DISPLAY_STEP == 0:

            loss_val, acc, s, label, pred, c_pred = sess.run([loss_op, accuracy_op, summary, y, prediction['classes'], correct_pred],
                                        feed_dict={keep_prob: 1.0})

            writer.add_summary(s, step)
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss_val) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    
    print('Done.')

Load Pretrained model
INFO:tensorflow:Restoring parameters from ./imagenet_pretrained_20/model.ckpt
Model restored.
Start training.
Step 0, Minibatch Loss= 3.5255, Training Accuracy= 0.094
Step 1, Minibatch Loss= 3.0487, Training Accuracy= 0.078
Step 2, Minibatch Loss= 2.9053, Training Accuracy= 0.141
Step 3, Minibatch Loss= 2.9598, Training Accuracy= 0.141
Step 4, Minibatch Loss= 2.8781, Training Accuracy= 0.117
Step 5, Minibatch Loss= 2.7582, Training Accuracy= 0.172
Step 6, Minibatch Loss= 2.7883, Training Accuracy= 0.102
Step 7, Minibatch Loss= 2.8787, Training Accuracy= 0.109
Step 8, Minibatch Loss= 2.8134, Training Accuracy= 0.148
Step 9, Minibatch Loss= 2.7855, Training Accuracy= 0.195
Step 10, Minibatch Loss= 2.7800, Training Accuracy= 0.133
Step 11, Minibatch Loss= 2.7348, Training Accuracy= 0.172
Step 12, Minibatch Loss= 2.7360, Training Accuracy= 0.125
Step 13, Minibatch Loss= 3.0137, Training Accuracy= 0.148
Step 14, Minibatch Loss= 2.9147, Training Accuracy= 0.102
Step 15,

In [10]:
with tf.Session(graph=model_graph) as sess:
    
    sess.run(global_init)
    
    alexnet.load_weights(session=sess, encoding='latin1')
    print('weights loaded.')
    
    # Save the model
    save_path = saver.save(sess, IMGNET_PRE_MODEL)
    print('Model saved in: %s' % save_path)

weights loaded.
Model saved in: ./imagenet_pretrained_20/model.ckpt
